In [33]:
import numpy as np
import pandas as pd
import random

In [34]:
df_interactions = pd.read_csv('interactions.csv')

In [36]:
df_interactions

,user_id,week_no,shares_deal
0,1,30,"[7681, 8498, 5529, 5351, 1174, 2026, 5907]"
1,1,34,"[5511, 3289, 7661, 8398]"
2,1,39,"[1274, 3627, 1833, 1, 2387, 7291]"
3,1,41,"[2661, 354]"
4,1,46,"[4383, 349, 2173, 7362, 4671, 7162]"
...,...,...,...
149995,25000,32,"[4633, 3494, 5524, 3498]"
149996,25000,38,"[3494, 279, 2142]"
149997,25000,43,[2757]
149998,25000,44,"[8154, 716]"


In [37]:
df_interactions.shape

(150000, 3)

In [38]:
df_shares = pd.read_csv('finviz_shares.csv')

In [39]:
df_shares.head()

,ticker,id_share,industry,Index,P/E,EPS (ttm),Insider Own,Perf Week,Forward P/E,EPS next Y,...,Market Cap_full_count,Income_full_count,Sales_full_count,Shs Outstand_full_count,Shs Float_full_count,Avg Volume_full_count,52_w_range_low,52_w_range_high,Volatility_low_%,Volatility_high_%
0,AAP,1,Consumer Cyclical,S&P 500,22.54,9.88,0.10,-4.20,16.78,11.37,...,1.423000e+13,6.464000e+08,1.097000e+13,6.285000e+07,6.208000e+07,679290.0,151.01,244.55,2.73,2.73
1,AAQC,2,Financial,-,NaN,NaN,NaN,-0.10,NaN,NaN,...,3.880000e+08,NaN,NaN,4.000000e+07,NaN,55940.0,9.60,9.96,0.16,0.18
2,AAOI,3,Technology,-,NaN,-2.05,6.40,2.08,NaN,15.40,...,1.045800e+08,-5.310000e+07,2.095000e+08,2.710000e+07,2.539000e+07,506560.0,3.63,12.97,5.01,5.43
3,AAPL,4,Technology,DJIA S&P500,28.58,6.02,0.07,-0.45,26.29,6.42,...,2.817210e+15,1.005600e+14,3.783200e+14,1.639000e+13,1.631000e+13,101620000.0,116.21,182.94,1.87,2.63
4,AAON,5,Industrials,-,43.79,1.33,16.60,-4.43,34.57,34.88,...,3.120000e+12,7.150000e+07,5.149000e+08,5.242000e+07,4.156000e+07,244130.0,58.65,83.79,3.30,3.88


In [40]:
df_interactions.shares_deal = df_interactions.shares_deal.astype(str).str.replace('[', '')

<ipython-input-40-79dd445d72ac>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df_interactions.shares_deal = df_interactions.shares_deal.astype(str).str.replace('[', '')


In [41]:
df_interactions.shares_deal = df_interactions.shares_deal.astype(str).str.replace(']', '')

<ipython-input-41-4bbf7af6bae3>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df_interactions.shares_deal = df_interactions.shares_deal.astype(str).str.replace(']', '')


In [42]:
df_interactions.shares_deal = df_interactions.shares_deal.map(lambda x: x.split(', '))

In [43]:
type(df_interactions.shares_deal[1])

list

In [44]:
df_interactions.tail()

,user_id,week_no,shares_deal
149995,25000,32,"[4633, 3494, 5524, 3498]"
149996,25000,38,"[3494, 279, 2142]"
149997,25000,43,[2757]
149998,25000,44,"[8154, 716]"
149999,25000,51,[5996]


In [45]:
def shares_treatment(feature_name, X):
    
    list_of_shares = X[feature_name]
    new_list = []
    
    for i in range(X.shape[0]):
        temp_list = []
        for j in range(len(list_of_shares[i])): 
            if list_of_shares[i][j].isdigit() == True: 
                temp_list.append(float(list_of_shares[i][j]))
        new_list.append(temp_list)
    
    X[feature_name] = new_list
    
    return X

In [46]:
df_interactions = shares_treatment(feature_name = 'shares_deal', X = df_interactions)

In [47]:
# кол-во покупок акций за 1 неделю
df_interactions['count_shares'] = df_interactions.shares_deal.map(lambda x: len(x))

In [52]:
df_interactions.head()

,user_id,week_no,count_shares,id_shares
0,1,30,7,7681.0
0,1,30,7,8498.0
0,1,30,7,5529.0
0,1,30,7,5351.0
0,1,30,7,1174.0


In [49]:
# "стакаем" номера недель 
s = df_interactions.apply(lambda x: pd.Series(x['shares_deal']), axis=1).stack().reset_index(level=1, drop=True)

In [50]:
s.name = 'id_shares'
df_interactions = df_interactions.drop('shares_deal', axis=1).join(s)

In [51]:
df_interactions.head(20)

,user_id,week_no,count_shares,id_shares
0,1,30,7,7681.0
0,1,30,7,8498.0
0,1,30,7,5529.0
0,1,30,7,5351.0
0,1,30,7,1174.0
0,1,30,7,2026.0
0,1,30,7,5907.0
1,1,34,4,5511.0
1,1,34,4,3289.0
1,1,34,4,7661.0


In [53]:
df_interactions = df_interactions.reset_index()

In [54]:
df_interactions = df_interactions.drop('index', axis=1)

In [55]:
df_interactions

,user_id,week_no,count_shares,id_shares
0,1,30,7,7681.0
1,1,30,7,8498.0
2,1,30,7,5529.0
3,1,30,7,5351.0
4,1,30,7,1174.0
...,...,...,...,...
601123,25000,38,3,2142.0
601124,25000,43,1,2757.0
601125,25000,44,2,8154.0
601126,25000,44,2,716.0


__Осталось добавить кол-во каждой акции при покупке. Будем диверсифицировать кол-во акций каждого инвестора в зависимости от отраслей__

In [56]:
df_shares['industry'].unique()

array(['Consumer Cyclical', 'Financial', 'Technology', 'Industrials',
       'Healthcare', 'Consumer Defensive', 'Real Estate',
       'Basic Materials', 'Communication Services', 'Utilities', 'Energy'],
      dtype=object)

In [57]:
industries = df_shares['industry'].to_list()

In [58]:
def industry_list_shares(industry, X):
    industry_df = X[X['industry'] == industry]
    industry_list = industry_df.id_share.to_list()
    industry_list = [float(i) for i in industry_list]

    return industry_list

In [59]:
financial_list = industry_list_shares('Financial', df_shares)
consumer_list = industry_list_shares('Consumer Cyclical', df_shares)
technology_list = industry_list_shares('Technology', df_shares)
industrial_list = industry_list_shares('Industrials', df_shares)
healthcare_list = industry_list_shares('Healthcare', df_shares)
defensive_list = industry_list_shares('Consumer Defensive', df_shares) 
estate_list = industry_list_shares('Real Estate', df_shares)
materials_list = industry_list_shares('Basic Materials', df_shares)
service_list = industry_list_shares('Communication Services', df_shares) 
utils_list = industry_list_shares('Utilities', df_shares)
energy_list = industry_list_shares('Energy', df_shares)

_Кол-во акций на каждую отрасль_

In [60]:
#id всех акций из датасета df_interactions: 
list_id_shares = df_interactions.id_shares.to_list()

In [61]:
quantity_2 = []
for sh in list_id_shares: 
    if sh in financial_list: 
        quantity_2.append(random.randint(260, 275))
    elif sh in consumer_list:
        quantity_2.append(random.randint(125, 150))
    elif sh in technology_list:
        quantity_2.append(random.randint(250, 260))
    elif sh in industrial_list:
        quantity_2.append(random.randint(15, 20))
    elif sh in healthcare_list:
        quantity_2.append(random.randint(225, 250))
    elif sh in defensive_list:
        quantity_2.append(random.randint(40, 65))
    elif sh in estate_list:
        quantity_2.append(random.randint(10, 15))
    elif sh in materials_list:
        quantity_2.append(random.randint(75, 100))
    elif sh in service_list:
        quantity_2.append(random.randint(1, 5))
    elif sh in utils_list:
        quantity_2.append(random.randint(5, 10))
    elif sh in energy_list:
        quantity_2.append(random.randint(175, 200))

In [62]:
df_interactions['quantity'] = quantity_2

In [63]:
#count shares - это кол-во купленных акций юзером id в определенную неделю. 
# например за 29 неделю пользователь номер 1 купил 2 акции. 
# опционально можно менять подходы: ставить или quantity или count_shares
df_interactions

,user_id,week_no,count_shares,id_shares,quantity
0,1,30,7,7681.0,263
1,1,30,7,8498.0,237
2,1,30,7,5529.0,49
3,1,30,7,5351.0,145
4,1,30,7,1174.0,134
...,...,...,...,...,...
601123,25000,38,3,2142.0,136
601124,25000,43,1,2757.0,16
601125,25000,44,2,8154.0,248
601126,25000,44,2,716.0,270


In [64]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from implicit.als import AlternatingLeastSquares
from implicit.bpr import BayesianPersonalizedRanking
from implicit.nearest_neighbours import bm25_weight

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

## Train-test split

In [65]:
test_size_weeks = 4

data_train = df_interactions[df_interactions['week_no'] < df_interactions['week_no'].max() - test_size_weeks]
data_test = df_interactions[df_interactions['week_no'] >= df_interactions['week_no'].max() - test_size_weeks]

## Метрика

In [66]:
def precision_at_k(recommended_list, bought_list):
    
    k=5
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    flags = np.isin(bought_list, recommended_list)
    #print(flags)
    precision = flags.sum() / len(recommended_list)
    
    return precision

_Для бизнеса и для ML я выбрал метрику `precision_at_k.` В первую очередь брокер заинтересован в том, чтобы клиент
совершал сделки, а во вторую очередь зарабатывал. Идеальным вариантом считаю кастомную метрику, которая отражает
доход с каждой рекомендации_

## Бейзлайны

Создадим датафрейм с покупками юзеров на тестовом датасете (последние 3 недели)

In [67]:
result = data_test.groupby('user_id')['id_shares'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[7463.0, 5423.0, 1855.0, 5931.0]"
1,2,"[1793.0, 3387.0, 3351.0, 5889.0, 30.0, 6241.0,..."


### 1.2 Popularity-based recommendation

In [68]:
def spb100_recommendation(shares, n=5):
    """Топ-n популярных акций из индекса spb-100"""
    
    recs = np.array(shares)
    recs = [float(i) for i in recs]
    rec_list = random.sample(recs, k=n)
    
    return rec_list

In [69]:
spb = pd.read_csv('popular_spb.csv')

In [70]:
%%time

shares_spb = spb.id_share.unique()

result['spb100_recommendation'] = result['user_id'].apply(lambda x: spb100_recommendation(shares_spb, n=5))
result.head(2)

Wall time: 1.13 s


,user_id,actual,spb100_recommendation
0,1,"[7463.0, 5423.0, 1855.0, 5931.0]","[4.0, 1833.0, 4624.0, 7745.0, 7661.0]"
1,2,"[1793.0, 3387.0, 3351.0, 5889.0, 30.0, 6241.0,...","[4792.0, 8040.0, 7392.0, 2553.0, 2331.0]"


In [71]:
def popularity_recommendation(data, n=5):
    """Топ-n рандомно популярных акций"""
    
    popular = data.groupby('id_shares')['quantity'].sum().reset_index()
    popular.sort_values('quantity', ascending=False, inplace=True)
    
    recs = popular.head(25).id_shares.to_list()
    
    rec_list = random.sample(recs, k=n)
    
    return rec_list

In [72]:
####

In [73]:
popular_recs = popularity_recommendation(data_train, n=5)

result['popular_recommendation'] = result['user_id'].apply(lambda x: popular_recs)
result.head(2)

,user_id,actual,spb100_recommendation,popular_recommendation
0,1,"[7463.0, 5423.0, 1855.0, 5931.0]","[4.0, 1833.0, 4624.0, 7745.0, 7661.0]","[6276.0, 6260.0, 5172.0, 5113.0, 4060.0]"
1,2,"[1793.0, 3387.0, 3351.0, 5889.0, 30.0, 6241.0,...","[4792.0, 8040.0, 7392.0, 2553.0, 2331.0]","[6276.0, 6260.0, 5172.0, 5113.0, 4060.0]"


__Сформируем датасет из популярных акций__

In [74]:
popularity = df_interactions.groupby('id_shares')['quantity'].sum().reset_index()

popularity.head(500)

,id_shares,quantity
0,1.0,151094
1,2.0,1851
2,3.0,2818
3,4.0,456810
4,5.0,72
...,...,...
495,501.0,4822
496,502.0,2403
497,503.0,68
498,504.0,1267


In [75]:
top_250 = popularity.sort_values('quantity', ascending=False).head(250).id_shares.tolist()

In [76]:
# Заведем фиктивный shares_id (если юзер не купил акцию из топ-250, то он "купил" такую акцию)
data_train.loc[~data_train['id_shares'].isin(top_250), 'id_shares'] = 999999

user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='id_shares', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix[user_item_matrix > 0] = 1 # так как в итоге хотим предсказать 
user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(3)

c:\users\anatoly\appdata\local\programs\python\python39\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


id_shares,4.0,19.0,28.0,30.0,39.0,43.0,95.0,104.0,118.0,130.0,...,8356.0,8367.0,8380.0,8435.0,8458.0,8498.0,8535.0,8537.0,8555.0,999999.0
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [77]:
user_item_matrix.shape

(25000, 251)

In [78]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

## ItemItemRecommender

In [79]:
%%time

model = ItemItemRecommender(K=5, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, # кол-во рекомендаций 
                        filter_already_liked_items=True, 
                        filter_items=None, 
                        recalculate_user=True)

  0%|          | 0/251 [00:00<?, ?it/s]

Wall time: 453 ms


In [80]:
result['itemitem'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=True, 
                                    filter_items=None, 
                                    recalculate_user=True)])

In [81]:
result.head(2)

,user_id,actual,spb100_recommendation,popular_recommendation,itemitem
0,1,"[7463.0, 5423.0, 1855.0, 5931.0]","[4.0, 1833.0, 4624.0, 7745.0, 7661.0]","[6276.0, 6260.0, 5172.0, 5113.0, 4060.0]","[388.0, 5604.0, 8458.0, 5747.0, 5172.0]"
1,2,"[1793.0, 3387.0, 3351.0, 5889.0, 30.0, 6241.0,...","[4792.0, 8040.0, 7392.0, 2553.0, 2331.0]","[6276.0, 6260.0, 5172.0, 5113.0, 4060.0]","[8458.0, 1833.0, 388.0, 5604.0, 7924.0]"


## Cosine Similarity

In [82]:
%%time

model = CosineRecommender(K=5, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(), 
          show_progress=True)

recs = model.recommend(userid=userid_to_id[1], 
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, 
                        filter_already_liked_items=True, 
                        filter_items=None, 
                        recalculate_user=False)

  0%|          | 0/251 [00:00<?, ?it/s]

Wall time: 436 ms


In [83]:
[id_to_itemid[rec[0]] for rec in recs]

[388.0, 5604.0, 8458.0, 5747.0, 436.0]

In [84]:
%%time

result['cosine'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=True, 
                                    filter_items=None, 
                                    recalculate_user=True)])

Wall time: 1.19 s


In [85]:
result.head(2)

,user_id,actual,spb100_recommendation,popular_recommendation,itemitem,cosine
0,1,"[7463.0, 5423.0, 1855.0, 5931.0]","[4.0, 1833.0, 4624.0, 7745.0, 7661.0]","[6276.0, 6260.0, 5172.0, 5113.0, 4060.0]","[388.0, 5604.0, 8458.0, 5747.0, 5172.0]","[388.0, 5604.0, 8458.0, 5747.0, 436.0]"
1,2,"[1793.0, 3387.0, 3351.0, 5889.0, 30.0, 6241.0,...","[4792.0, 8040.0, 7392.0, 2553.0, 2331.0]","[6276.0, 6260.0, 5172.0, 5113.0, 4060.0]","[8458.0, 1833.0, 388.0, 5604.0, 7924.0]","[1833.0, 388.0, 8458.0, 5604.0, 7924.0]"


## TFIDFRecommender

In [86]:
%%time

model = TFIDFRecommender(K=5, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(), 
          show_progress=True)

recs = model.recommend(userid=userid_to_id[1], 
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, 
                        filter_already_liked_items=True, 
                        filter_items=None, 
                        recalculate_user=False)

  0%|          | 0/251 [00:00<?, ?it/s]

Wall time: 453 ms


In [87]:
[id_to_itemid[rec[0]] for rec in recs]

[5604.0, 5889.0, 8458.0, 716.0, 5747.0]

In [88]:
%%time

result['tfidf'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=True, 
                                    filter_items=None, 
                                    recalculate_user=False)])

Wall time: 1.17 s


In [89]:
result.head(2)

,user_id,actual,spb100_recommendation,popular_recommendation,itemitem,cosine,tfidf
0,1,"[7463.0, 5423.0, 1855.0, 5931.0]","[4.0, 1833.0, 4624.0, 7745.0, 7661.0]","[6276.0, 6260.0, 5172.0, 5113.0, 4060.0]","[388.0, 5604.0, 8458.0, 5747.0, 5172.0]","[388.0, 5604.0, 8458.0, 5747.0, 436.0]","[5604.0, 5889.0, 8458.0, 716.0, 5747.0]"
1,2,"[1793.0, 3387.0, 3351.0, 5889.0, 30.0, 6241.0,...","[4792.0, 8040.0, 7392.0, 2553.0, 2331.0]","[6276.0, 6260.0, 5172.0, 5113.0, 4060.0]","[8458.0, 1833.0, 388.0, 5604.0, 7924.0]","[1833.0, 388.0, 8458.0, 5604.0, 7924.0]","[8458.0, 5604.0, 5889.0, 716.0, 7924.0]"


## ALS

In [90]:
%%time

model = AlternatingLeastSquares(factors=100,  # Нужно подбирать
                                regularization=0.001,
                                iterations=15,
                                calculate_training_loss=True, 
                                num_threads=4)

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, # кол-во рекомендаций 
                        filter_already_liked_items=[itemid_to_id[999999]], 
                        filter_items=None, 
                        recalculate_user=False)

  0%|          | 0/15 [00:00<?, ?it/s]

Wall time: 31.3 s


In [91]:
recs

[(236, 0.21963489),
 (47, 0.21725845),
 (165, 0.20511593),
 (84, 0.19392784),
 (104, 0.17844523)]

In [92]:
[id_to_itemid[rec[0]] for rec in recs]

[8235.0, 1238.0, 5339.0, 2458.0, 3387.0]

In [93]:
def get_recommendations(user, model, N=5):
    res = [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[user], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=N, 
                                    filter_already_liked_items=False, 
                                    filter_items=[itemid_to_id[999999]], 
                                    recalculate_user=True)]
    return res

In [94]:
%%time
    
result['als'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))

Wall time: 50.8 s


In [95]:
result.head(2)

,user_id,actual,spb100_recommendation,popular_recommendation,itemitem,cosine,tfidf,als
0,1,"[7463.0, 5423.0, 1855.0, 5931.0]","[4.0, 1833.0, 4624.0, 7745.0, 7661.0]","[6276.0, 6260.0, 5172.0, 5113.0, 4060.0]","[388.0, 5604.0, 8458.0, 5747.0, 5172.0]","[388.0, 5604.0, 8458.0, 5747.0, 436.0]","[5604.0, 5889.0, 8458.0, 716.0, 5747.0]","[1833.0, 354.0, 5511.0, 4383.0, 349.0]"
1,2,"[1793.0, 3387.0, 3351.0, 5889.0, 30.0, 6241.0,...","[4792.0, 8040.0, 7392.0, 2553.0, 2331.0]","[6276.0, 6260.0, 5172.0, 5113.0, 4060.0]","[8458.0, 1833.0, 388.0, 5604.0, 7924.0]","[1833.0, 388.0, 8458.0, 5604.0, 7924.0]","[8458.0, 5604.0, 5889.0, 716.0, 7924.0]","[1703.0, 6241.0, 5270.0, 95.0, 8498.0]"


In [96]:
## Попробуем применить bm-25 взвешивание

In [97]:
user_item_matrix = bm25_weight(user_item_matrix.T).T  # Применяется к item-user матрице ! 

In [98]:
%%time

model = AlternatingLeastSquares(factors=100, 
                                regularization=0.001,
                                iterations=15, 
                                calculate_training_loss=True, 
                                num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

result['als_bm25'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))

  0%|          | 0/15 [00:00<?, ?it/s]

Wall time: 1min 31s


In [100]:
result.head()

,user_id,actual,spb100_recommendation,popular_recommendation,itemitem,cosine,tfidf,als,als_bm25
0,1,"[7463.0, 5423.0, 1855.0, 5931.0]","[4.0, 1833.0, 4624.0, 7745.0, 7661.0]","[6276.0, 6260.0, 5172.0, 5113.0, 4060.0]","[388.0, 5604.0, 8458.0, 5747.0, 5172.0]","[388.0, 5604.0, 8458.0, 5747.0, 436.0]","[5604.0, 5889.0, 8458.0, 716.0, 5747.0]","[1833.0, 354.0, 5511.0, 4383.0, 349.0]","[1833.0, 354.0, 7681.0, 5511.0, 349.0]"
1,2,"[1793.0, 3387.0, 3351.0, 5889.0, 30.0, 6241.0,...","[4792.0, 8040.0, 7392.0, 2553.0, 2331.0]","[6276.0, 6260.0, 5172.0, 5113.0, 4060.0]","[8458.0, 1833.0, 388.0, 5604.0, 7924.0]","[1833.0, 388.0, 8458.0, 5604.0, 7924.0]","[8458.0, 5604.0, 5889.0, 716.0, 7924.0]","[1703.0, 6241.0, 5270.0, 95.0, 8498.0]","[1703.0, 6241.0, 95.0, 5270.0, 5817.0]"
2,3,"[7681.0, 2767.0, 2173.0, 938.0, 1539.0, 6334.0...","[4792.0, 8058.0, 6276.0, 8544.0, 8458.0]","[6276.0, 6260.0, 5172.0, 5113.0, 4060.0]","[388.0, 5604.0, 8458.0, 1833.0, 7664.0]","[5604.0, 1833.0, 388.0, 8458.0, 30.0]","[5604.0, 5889.0, 8458.0, 716.0, 7664.0]","[5747.0, 3611.0, 1308.0, 7658.0, 349.0]","[349.0, 7658.0, 1770.0, 3611.0, 5747.0]"
3,4,"[6037.0, 7937.0]","[699.0, 1124.0, 3325.0, 4287.0, 716.0]","[6276.0, 6260.0, 5172.0, 5113.0, 4060.0]","[8458.0, 1833.0, 5604.0, 388.0, 7664.0]","[8458.0, 5604.0, 388.0, 1833.0, 7664.0]","[8458.0, 5604.0, 5889.0, 716.0, 7664.0]","[8154.0, 7498.0, 2806.0, 3366.0, 1982.0]","[2806.0, 8154.0, 7498.0, 2995.0, 3366.0]"
4,5,"[4066.0, 2803.0, 5297.0, 388.0, 4423.0, 7362.0...","[6260.0, 5172.0, 4252.0, 3819.0, 7392.0]","[6276.0, 6260.0, 5172.0, 5113.0, 4060.0]","[1833.0, 8458.0, 5604.0, 388.0, 5747.0]","[1833.0, 5604.0, 8458.0, 388.0, 6276.0]","[5604.0, 716.0, 8458.0, 5889.0, 6276.0]","[4060.0, 354.0, 2700.0, 4092.0, 4056.0]","[354.0, 4060.0, 4056.0, 4882.0, 2700.0]"


## Bayesian personalized ranking

In [101]:
## Попробуем применить bm-25 взвешивание + BayesianPersonalizedRanking

In [102]:
%%time

model_bpr = BayesianPersonalizedRanking(factors=20, 
                                regularization=0.001,
                                iterations=15,  
                                num_threads=4)

model_bpr.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix, взвешенную по bm25
          show_progress=True)

  0%|          | 0/15 [00:00<?, ?it/s]

Wall time: 1.05 s


In [103]:
def get_recommendations(user, model, N=5):
    res = [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[user], 
                                    user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                                    N=N, 
                                    filter_already_liked_items=False, 
                                    filter_items=[itemid_to_id[999999]],  # !!! 
                                    recalculate_user=False)]
    return res

In [104]:
result['bm25_bpr'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model_bpr, N=5))

In [105]:
# Посмотрим результаты по метрике

In [106]:
pd.DataFrame([
    result.apply(lambda row: precision_at_k(row['spb100_recommendation'], row['actual']), axis=1).mean(),
    result.apply(lambda row: precision_at_k(row['popular_recommendation'], row['actual']), axis=1).mean(),
    result.apply(lambda row: precision_at_k(row['itemitem'], row['actual']), axis=1).mean(),
    result.apply(lambda row: precision_at_k(row['cosine'], row['actual']), axis=1).mean(),
    result.apply(lambda row: precision_at_k(row['tfidf'], row['actual']), axis=1).mean(),
    result.apply(lambda row: precision_at_k(row['als'], row['actual']), axis=1).mean(),
    result.apply(lambda row: precision_at_k(row['als_bm25'], row['actual']), axis=1).mean(),
    result.apply(lambda row: precision_at_k(row['bm25_bpr'], row['actual']), axis=1).mean(),
], 
    columns=['precision_at_k'], 
    index=['spb100_recommendation', 'popular_recommendation', 'itemitem', 'cosine', 'tfidf', 'als',
          'als_bm25', 'bm25_bpr']
).T

,spb100_recommendation,popular_recommendation,itemitem,cosine,tfidf,als,als_bm25,bm25_bpr
precision_at_k,0.01044,0.014536,0.01528,0.015112,0.014928,0.011104,0.010672,0.010784


In [78]:
# Метрики, конечно, удивили.... 
# - Как улучшить/изменить ситуацию (сузить круг бумаг или круг пользователей): 
#        - изменить снова входные id_share. Поставить предфильтрацию по наличию бумаги в s&p500
#        - изменить кол-во уникальных пользователей. Сократить со 100 000 до 2500 и провести снова тест

In [107]:
# Было: 
#       spb100_recommendation	popular_recommendation	itemitem	cosine	tfidf	als	als_bm25	bm25_bpr

# precision_at_k	0.00048	0.000452	0.000422	0.000418	0.000422	0.000526	0.00063	0.000586

In [108]:
## Метрики улучшились в разы из-за изменения распределения по акциям, но по-прежнему все не очень хорошо. 1.5% для рекомендательной
# системы - очень плохая история. 
# Надо убирать 'рандомную' акцию из листа всех акций и настраивать первичную предфильтрацию. Например, отфильтровать сначала только
# уботочные компании. 